<a href="https://colab.research.google.com/github/Athugodage/RuLawSimplification/blob/main/two_way%5Ctext_inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 5.3 MB/s 
     |████████████████████████████████| 7.6 MB 49.7 MB/s 
     |████████████████████████████████| 182 kB 57.0 MB/s 


In [2]:
import torch
import transformers
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import pandas as pd
import json

In [1]:
!wget https://github.com/Athugodage/RuLawSimplification/blob/33055914640e40b3a405fae69c4caa81012586a5/full_corpus.zip

--2022-12-16 12:46:35--  https://github.com/Athugodage/RuLawSimplification/blob/33055914640e40b3a405fae69c4caa81012586a5/full_corpus.zip
Resolving github.com (github.com)... 140.82.112.3
Connecting to github.com (github.com)|140.82.112.3|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘full_corpus.zip’

full_corpus.zip         [ <=>                ] 136.73K  --.-KB/s    in 0.01s   

2022-12-16 12:46:35 (10.8 MB/s) - ‘full_corpus.zip’ saved [140016]



In [3]:
!unzip 'corpus.zip'


Archive:  corpus.zip
  inflating: full_corpus.csv         


Теперь, после проверки, можно начать

In [13]:
data = pd.read_csv('full_corpus.csv', encoding='utf-8')
data

,Название документа,Ссылка,Текст,Комментарий РГ,Дата
0,"Федеральный закон от 28 мая 2022 г. N 146-ФЗ ""...",https://rg.ru/documents/2022/05/30/146biudzhet...,Принят Государственной Думой 24 мая 2022 года\...,"Объем кредитов, которые будут выданы бюджетам ...",30 мая 2022
1,Федеральный закон от 14 июля 2022 г. N 316-Ф3 ...,https://rg.ru/documents/2022/07/21/document-ze...,Принят Государственной Думой 6 июля 2022 года\...,Фермеры смогут получить в аренду государственн...,21 июля 2022
2,Федеральный закон от 14 июля 2022 г. N 271-ФЗ ...,https://rg.ru/documents/2022/07/20/dokument-te...,Принят Государственной Думой 29 июня 2022 года...,Управляющие компании теперь могут проводить ко...,20 июля 2022
3,Федеральный закон от 14 июля 2022 г. N 290-ФЗ ...,https://rg.ru/documents/2022/07/20/dokument-ko...,Принят Государственной Думой 6 июля 2022 года\...,Поправки в КоАП распространяют на бизнес так н...,20 июля 2022
4,Федеральный закон от 14 июля 2022 г. N 287-ФЗ ...,https://rg.ru/documents/2022/07/20/document-ko...,Принят Государственной Думой 29 июня 2022 года...,Выброс мусора из автомобиля теперь попадет в к...,20 июля 2022
...,...,...,...,...,...
2022,"Утверждены правила предоставления субсидий ""РЖ...",https://rg.ru/2009/03/11/postanovlenie-dok.html,В соответствии с частью 6 статьи 25 Федерально...,Утверждены правила предоставления в 2009 году ...,11 марта 2009
2023,Правила перевозок пассажиров и багажа обществе...,https://rg.ru/2009/03/11/bagaj-dok.html,В соответствии со статьей 3 Федерального закон...,В России впервые за 22 года утвердили новые пр...,11 марта 2009
2024,Изменения в правила направления средств матери...,https://rg.ru/2009/01/16/matkapital-jilie-dok....,Правительство Российской Федерации постановляе...,Правительство утвердило изменения в правила на...,16 января 2009
2025,"Федеральный закон ""О государственной регистрац...",https://rg.ru/2009/03/17/gosregistraciya-dok.html,Принят Государственной Думой 25 февраля 2009 г...,Все воздушные суда теперь подлежат обязательно...,17 марта 2009


In [5]:

text = [full_text for full_text in data['Текст']]
comment = [full_com for full_com in data['Комментарий РГ']]

In [6]:
model_checkpoint = 'cointegrated/rubert-base-cased-nli-twoway'
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint)
if torch.cuda.is_available():
    model.cuda()


Downloading:   0%|          | 0.00/545 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.65M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.62M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.11k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/712M [00:00<?, ?B/s]

In [7]:
def exchange(com, tekst):   
    
   
    with torch.inference_mode():
        out = model(**tokenizer(com, tekst, padding=True, truncation=True,max_length=50, add_special_tokens = True, return_tensors='pt').to(model.device))
        proba = torch.softmax(out.logits, -1).cpu().numpy()[0]
        x1 = {v: proba[k] for k, v in model.config.id2label.items()}
    with torch.inference_mode():
        out = model(**tokenizer(tekst, com, padding=True, truncation=True,max_length=50, add_special_tokens = True, return_tensors='pt').to(model.device))
        proba = torch.softmax(out.logits, -1).cpu().numpy()[0]
        x2 = {v: proba[k] for k, v in model.config.id2label.items()}

    x = x1['entailment'] + x2['entailment']
    return x

In [ ]:
type(text[0])

str

In [8]:
from tqdm import tqdm

scores = []

for n in tqdm(range(len(text))):
    article = text[n]
    simple = comment[n]
    try:
        score = exchange(simple, article)
    except:
        score = None
    
    scores.append(score)




  

100%|██████████| 2027/2027 [39:44<00:00,  1.18s/it]


In [15]:
data['scores'] = scores

In [16]:
data

,Название документа,Ссылка,Текст,Комментарий РГ,Дата,scores
0,"Федеральный закон от 28 мая 2022 г. N 146-ФЗ ""...",https://rg.ru/documents/2022/05/30/146biudzhet...,Принят Государственной Думой 24 мая 2022 года\...,"Объем кредитов, которые будут выданы бюджетам ...",30 мая 2022,0.014911
1,Федеральный закон от 14 июля 2022 г. N 316-Ф3 ...,https://rg.ru/documents/2022/07/21/document-ze...,Принят Государственной Думой 6 июля 2022 года\...,Фермеры смогут получить в аренду государственн...,21 июля 2022,0.007019
2,Федеральный закон от 14 июля 2022 г. N 271-ФЗ ...,https://rg.ru/documents/2022/07/20/dokument-te...,Принят Государственной Думой 29 июня 2022 года...,Управляющие компании теперь могут проводить ко...,20 июля 2022,0.012755
3,Федеральный закон от 14 июля 2022 г. N 290-ФЗ ...,https://rg.ru/documents/2022/07/20/dokument-ko...,Принят Государственной Думой 6 июля 2022 года\...,Поправки в КоАП распространяют на бизнес так н...,20 июля 2022,0.016304
4,Федеральный закон от 14 июля 2022 г. N 287-ФЗ ...,https://rg.ru/documents/2022/07/20/document-ko...,Принят Государственной Думой 29 июня 2022 года...,Выброс мусора из автомобиля теперь попадет в к...,20 июля 2022,0.011986
...,...,...,...,...,...,...
2022,"Утверждены правила предоставления субсидий ""РЖ...",https://rg.ru/2009/03/11/postanovlenie-dok.html,В соответствии с частью 6 статьи 25 Федерально...,Утверждены правила предоставления в 2009 году ...,11 марта 2009,0.161472
2023,Правила перевозок пассажиров и багажа обществе...,https://rg.ru/2009/03/11/bagaj-dok.html,В соответствии со статьей 3 Федерального закон...,В России впервые за 22 года утвердили новые пр...,11 марта 2009,0.262145
2024,Изменения в правила направления средств матери...,https://rg.ru/2009/01/16/matkapital-jilie-dok....,Правительство Российской Федерации постановляе...,Правительство утвердило изменения в правила на...,16 января 2009,0.531573
2025,"Федеральный закон ""О государственной регистрац...",https://rg.ru/2009/03/17/gosregistraciya-dok.html,Принят Государственной Думой 25 февраля 2009 г...,Все воздушные суда теперь подлежат обязательно...,17 марта 2009,0.009236


In [17]:
data.to_csv('with_scores.csv', index=False)

In [24]:
data[data['scores'] < 0.005]

,Название документа,Ссылка,Текст,Комментарий РГ,Дата,scores
6,Федеральный закон от 14 июля 2022 г. N 255-ФЗ ...,https://rg.ru/documents/2022/07/19/document-in...,Принят Государственной Думой 29 июня 2022 года...,"Законодательство в России, регулирующее деятел...",19 июля 2022,0.004243
22,Федеральный закон от 20 октября 2022 г. N 410-...,https://rg.ru/documents/2022/10/23/federalnyj-...,Принят Государственной Думой 18 октября 2022 г...,Наказание за нарушения правил борьбы с опасным...,23 октября 2022,0.004886
26,Федеральный закон от 14 июля 2022 г. N 260-ФЗ ...,https://rg.ru/documents/2022/07/19/document-uk...,Принят Государственной Думой 6 июля 2022 года\...,"Сегодня ""Российская газета"" публикует поправки...",19 июля 2022,0.004574
28,Федеральный закон от 14 июля 2022 года N 303-Ф...,https://rg.ru/documents/2022/07/21/document-ko...,Принят Государственной Думой 5 июля 2022 года\...,"В России будет создан единый банк данных, соде...",21 июля 2022,0.004093
35,Постановление Пленума Верховного Суда Российск...,https://rg.ru/2022/01/14/postanovlenie-dok.html,В целях единообразного применения судами полож...,Соответствующее постановление сегодня публикуе...,14 января 2022,0.004101
...,...,...,...,...,...,...
2011,Приказ Минздравсоцразвития об организации и ос...,https://rg.ru/2009/06/03/lekarstva-tseny-dok.html,Зарегистрирован в Минюсте РФ 28 мая 2009 г. Р...,Росздравнадзор начинает постоянный мониторинг ...,3 июня 2009,0.002635
2012,Гигиенические требования к организациям химиче...,https://rg.ru/2009/06/03/himchistka-dok.html,Зарегистрирован в Минюсте РФ 27 апреля 2009 г....,Химчистки теперь могут открываться не только в...,3 июня 2009,0.001673
2016,Технический регламент о безопасности зданий и ...,https://rg.ru/2009/12/31/tehreg-zdaniya-dok.html,Принят Государственной Думой 23 декабря 2009 г...,В России наконец появился технический регламен...,31 декабря 2009,0.004925
2019,Требования к оказанию услуг связи в части уста...,https://rg.ru/2009/02/27/minsvyaz-nomer-dok.html,Зарегистрирован в Минюсте РФ 12 февраля 2009 г...,С 1 апреля 2009 года при звонках в другие горо...,27 февраля 2009,0.002895


In [25]:
data['Текст'][6]

'Принят Государственной Думой 29 июня 2022 года\n  Одобрен Советом Федерации 8 июля 2022 года\n  Статья 1. Иностранные агенты\n  1. Для целей настоящего Федерального закона под иностранным агентом понимается лицо, получившее поддержку и (или) находящееся под иностранным влиянием в иных формах и осуществляющее деятельность, виды которой установлены статьей 4 настоящего Федерального закона.\n  2. Иностранным агентом может быть признано российское или иностранное юридическое лицо независимо от его организационно-правовой формы, общественное объединение, действующее без образования юридического лица, иное объединение лиц, иностранная структура без образования юридического лица, а также физическое лицо независимо от его гражданства или при отсутствии такового (далее - лица).\n  3. Иностранными агентами не могут быть признаны:\n  1) органы публичной власти Российской Федерации, лица, подконтрольные Российской Федерации, субъектам Российской Федерации, муниципальным образованиям, публично-пра

In [26]:
data['Комментарий РГ'][6]

'Законодательство в России, регулирующее деятельность иностранных агентов, отныне систематизировано, и в нем четко определены все понятия. Это предусматривает новый Закон "О контроле за деятельностью лиц, находящихся под иностранным влиянием". Его публикует "РГ", а в силу он вступит с 1 декабря 2022 года.\nЗакон устанавливает запрет влияния иноагентов на российских детей в любой форме. Иноагентам запрещено осуществлять преподавательскую, просветительскую, воспитательную деятельность в отношении несовершеннолетних, а также производить информационную продукцию для детей и подростков.\nВ документе уточняются сами понятия "иностранный агент", "иностранное влияние", "иностранный источник", "политическая деятельность". Также вводится единый реестр иноагентов и создается реестр тех, кто аффилирован с иностранными агентами. Его будет вести минюст. Ограничения, применяемые для иноагентов, на аффилированных лиц распространяться не будут.\nЗакон запрещает иноагентам получать в России госфинансиро

In [28]:
data.agg({'scores': ['min', 'mean', 'max']})

,scores
min,0.001213
mean,0.042761
max,1.265589


In [41]:
data['Комментарий РГ'][2023]

'В России впервые за 22 года утвердили новые правила перевозки пассажиров на автобусах, троллейбусах, трамваях и такси. Как пользоваться городским общественным транспортом, где узнать об изменениях в расписании и куда обратиться за забытыми вещами – уточняет Постановление Правительства. Документ опубликован в Российской газете.  Новые правила перевозки пассажиров отменяют норму, по которой на одном маршруте устанавливался не менее чем 20-минутный интервал между автобусами. Правда, от расписания регулярных рейсов авторы документа не отказались: его будут составлять и вывешивать на каждой остановке.  В новых правилах впервые прописана ответственность частных компаний-перевозчиков. Установлены четкие критерии для автомобилей такси: это "шашечки" на борту и оранжевый фонарь на крыше, который включается при готовности легкового такси к перевозке пассажира. На передней панели перед сиденьем пассажира должна быть подробная информация об условиях проезда: название таксопарка, адреса и телефоны

In [42]:
data['Текст'][2023]

'В соответствии со статьей 3 Федерального закона "Устав автомобильного транспорта и городского наземного электрического транспорта" Правительство Российской Федерации постановляет:  1. Утвердить прилагаемые Правила перевозок пассажиров и багажа автомобильным транспортом и городским наземным электрическим транспортом.  2. Признать утратившими силу:  Устав автомобильного транспорта РСФСР, утвержденный постановлением Совета Министров РСФСР от 8 января 1969 г. N12 (СП РСФСР, 1969, N 2 и 3, ст. 8);  постановление Совета Министров РСФСР от 28 ноября 1969 г. N 648 "Об изменении и признании утратившими силу решений Правительства РСФСР в связи с реорганизацией Министерства автомобильного транспорта и шоссейных дорог РСФСР" (СП РСФСР, 1969, N26, ст. 141);  постановление Совета Министров РСФСР от 17 сентября 1974 г. N 510 "О материальной ответственности автотранспортных предприятий и организаций, грузоотправителей и грузополучателей за ненадлежащее использование контейнеров при перевозках грузов 

In [43]:
data

,Название документа,Ссылка,Текст,Комментарий РГ,Дата,scores
0,"Федеральный закон от 28 мая 2022 г. N 146-ФЗ ""...",https://rg.ru/documents/2022/05/30/146biudzhet...,Принят Государственной Думой 24 мая 2022 года\...,"Объем кредитов, которые будут выданы бюджетам ...",30 мая 2022,0.014911
1,Федеральный закон от 14 июля 2022 г. N 316-Ф3 ...,https://rg.ru/documents/2022/07/21/document-ze...,Принят Государственной Думой 6 июля 2022 года\...,Фермеры смогут получить в аренду государственн...,21 июля 2022,0.007019
2,Федеральный закон от 14 июля 2022 г. N 271-ФЗ ...,https://rg.ru/documents/2022/07/20/dokument-te...,Принят Государственной Думой 29 июня 2022 года...,Управляющие компании теперь могут проводить ко...,20 июля 2022,0.012755
3,Федеральный закон от 14 июля 2022 г. N 290-ФЗ ...,https://rg.ru/documents/2022/07/20/dokument-ko...,Принят Государственной Думой 6 июля 2022 года\...,Поправки в КоАП распространяют на бизнес так н...,20 июля 2022,0.016304
4,Федеральный закон от 14 июля 2022 г. N 287-ФЗ ...,https://rg.ru/documents/2022/07/20/document-ko...,Принят Государственной Думой 29 июня 2022 года...,Выброс мусора из автомобиля теперь попадет в к...,20 июля 2022,0.011986
...,...,...,...,...,...,...
2022,"Утверждены правила предоставления субсидий ""РЖ...",https://rg.ru/2009/03/11/postanovlenie-dok.html,В соответствии с частью 6 статьи 25 Федерально...,Утверждены правила предоставления в 2009 году ...,11 марта 2009,0.161472
2023,Правила перевозок пассажиров и багажа обществе...,https://rg.ru/2009/03/11/bagaj-dok.html,В соответствии со статьей 3 Федерального закон...,В России впервые за 22 года утвердили новые пр...,11 марта 2009,0.262145
2024,Изменения в правила направления средств матери...,https://rg.ru/2009/01/16/matkapital-jilie-dok....,Правительство Российской Федерации постановляе...,Правительство утвердило изменения в правила на...,16 января 2009,0.531573
2025,"Федеральный закон ""О государственной регистрац...",https://rg.ru/2009/03/17/gosregistraciya-dok.html,Принят Государственной Думой 25 февраля 2009 г...,Все воздушные суда теперь подлежат обязательно...,17 марта 2009,0.009236
